In [1]:
"""
Mark Jay 코드 기반으로 함.
Notebook for streaming data from a microphone in realtime

audio is captured using pyaudio
then converted from binary data to ints using struct
then displayed using matplotlib

scipy.fftpack computes the FFT
"""

import pyaudio
import os
import struct
import numpy as np
import matplotlib.pyplot as plt
from scipy.fftpack import fft
import time
from tkinter import TclError

# to display in separate Tk window
%matplotlib tk

# constants
CHUNK = 2205   # RATE/200    # samples per frame, 프레임 
FORMAT = pyaudio.paInt16     # audio format (bytes per sample?)
CHANNELS = 1                 # single channel for microphone
RATE = 44100                 # samples per second
SECOND_PER_FRAME = CHUNK / RATE  # second per frame, 한 프레임당 걸리는 시간  

In [4]:

t = np.array(np.linspace(0, SECOND_PER_FRAME, CHUNK)) # len(t) = CHUNK
frequency = np.array(np.arange(0, 2*RATE, 1/SECOND_PER_FRAME ))  # len(frequencty) = CHUNK

len(t), len(frequency)

(2205, 4410)

In [5]:
# create matplotlib figure and axes
fig, (ax1, ax2) = plt.subplots(4, figsize=(15, 7))

# pyaudio class instance
p = pyaudio.PyAudio()

# stream object to get data from microphone
stream = p.open(
    format=FORMAT,
    channels=CHANNELS,
    rate=RATE,
    input=True,
    output=True,
    frames_per_buffer= CHUNK
)

# create a line object with random data 실시간 plot을 위한 line 함수들 
line, = ax1.plot(t, np.random.rand(CHUNK), '-', lw=2)  # CHUNK갯수 만큼의 rand(0~1 사이의 랜덤 숫자 생성) 행렬을 만든다. 

line_fft, = ax2.plot(frequency, np.random.rand(CHUNK), '-', lw=2)  # 그냥 

# format waveform axes
ax1.set_ylim(0, 255)
ax1.set_xlim(0, SECOND_PER_FRAME)

# format spectrum axes

ax2.set_xlim(20, RATE)
# ax2.set_ylim(0, 200000)       # 새로 추가 

while True:
    
    # binary data
    data = stream.read(CHUNK)  
    
    # convert data to integers, make np array, then offset it by 127
    data_int = struct.unpack(str(2 * CHUNK) + 'B', data)
    
    # create np array and offset by 128
    # data_np = np.array(data_int, dtype='b')[::2] + 128 

    data_np = np.array(data_int, dtype='b')[::2] + 128 
    
    line.set_ydata(data_np) # line(ax1)dp y데이터 입력 
    
    # compute FFT and update line
    yf = fft(data_int)
    line_fft.set_ydata(np.abs(yf)  / (128 * CHUNK))  # line_fft(ax2)에 y데이터 입력 
    # line_fft.set_ydata(np.abs(yf[0:CHUNK])  / 1 )             # line_fft(ax2)에 y데이터 입력 
    # update figure canvas
    
    fig.canvas.draw()
    fig.canvas.flush_events()


ValueError: too many values to unpack (expected 2)